In [ ]:
'''
This notebook is how to evaluate CNN Model which is built upon MFCC Numerica values. 
A. Evaluation against good and bad (noise) signals
B. Evaluation aginst mixed signal
C. Evaluation against Spear data
'''

In [15]:
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import numpy as np
import math
import pandas as pd
import os
import glob
import librosa
import numpy as np
from keras import models
import pickle

In [16]:
max_pad_len = 174

def extract_features_with_padding(audio, sample_rate):
    try:
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        pad_width = max_pad_len - mfccs.shape[1]
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')
        
    except Exception as e:
        print("Error encountered while parsing file: ")
        return None 
     
    return mfccs

In [6]:
def get_noise_from_sound(signal,noise,SNR):
    try:
        RMS_s=math.sqrt(np.mean(signal**2))
        #required RMS of noise
        RMS_n=math.sqrt(RMS_s**2/(pow(10,SNR/20)))

        #current RMS of noise
        RMS_n_current=math.sqrt(np.mean(noise**2))
        noise=noise*(RMS_n/RMS_n_current)
    
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None
    
    return noise

In [7]:
def process_sound(file_path):
    try:
        audio, sample_rate = librosa.load(file_path, sr=22050, res_type='kaiser_fast') 
        trim_signal = librosa.effects.trim(audio)
        total_duration = 4*sample_rate
        split_signal = trim_signal[0]
        if len(trim_signal[0]) > total_duration:
            split_signal=trim_signal[0][0:total_duration]
        
        aud_feature = extract_features_with_padding(split_signal, sample_rate)
        
    except Exception as e:
        print(e)
        return None
    
    return aud_feature

In [13]:
def validateAgainstMixData():
    parent_dir = '/Models/audio_data'
    sub_dirs= ['noise', 'good']
    sub_sub_good_dirs= ['flicker', 'spear_good', 'urban']
    sub_sub_noise_dirs= ['audio_mixed_noise', 'spear_noise', 'only_noise']
    file_ext = "*.wav"
    
    features = []

    for l, sub_dir in enumerate(sub_dirs):
        print('Processing folder: ', sub_dir)
        if sub_dir == 'good':
            processed_sub_dir = sub_sub_good_dirs
        
        if sub_dir == 'noise':
            processed_sub_dir = sub_sub_noise_dirs   
            
        for l, sub_sub_dir in enumerate(processed_sub_dir):
            
            for fn in glob.glob(os.path.join(parent_dir, sub_dir, sub_sub_dir, file_ext)):

                feature = process_sound(fn)

                if sub_dir == 'noise':
                    features.append([feature, 'noise'])

                if sub_dir == 'good':
                    features.append([feature, 'good'])

    return features

In [1]:
#model = tf.keras.models.load_model('/content/drive/My Drive/Image_Quality_Analysis/model/ImageQuality_TL_GD_V1_0408.h5',custom_objects={'KerasLayer':hub.KerasLayer})
model = models.load_model('/Models/CNN-MFCC-0418/SoundClassification_model_With_good_noise_V1_04182020_v1.h5')
model.load_weights('SoundClassification_Weight_With_good_noise_V1_04182020_v1.h5')
model.summary()

Using TensorFlow backend.


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 39, 173, 16)       80        
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 19, 86, 16)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 19, 86, 16)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 18, 85, 32)        2080      
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 9, 42, 32)         0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 9, 42, 32)         0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 8, 41, 64)        

In [3]:
with open('/Models/CNN-MFCC-0418/labelEncoderFile_With_good_noise_LE_04182020_v1.pkl', 'rb') as fid:
    le_loaded = pickle.load(fid)
le_loaded.classes_

D:\Users\618757\Anaconda3\envs\tensorFlowEnv\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.22.2.post1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


array(['good', 'noise'], dtype='<U5')

In [17]:
# This Section is to evaluate model agaisnt Good/Noise data

features = validateAgainstMixData()

print('total feature lenght: ' , len(features))
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
actualresult_result = np.array(featuresdf.class_label.tolist())

X = np.array(featuresdf.feature.tolist())
X = np.expand_dims(X, axis=3)

predict = model.predict(X)
predict=np.round(predict,decimals=5)
result = map(lambda v : np.argmax(v), predict)
prediction_result = np.array(list(result))
prediction_result = le_loaded.inverse_transform(prediction_result)
print(prediction_result)


classification_report = metrics.classification_report(actualresult_result, prediction_result)
print("Classification report: \n", classification_report)
confusion_matrix = metrics.confusion_matrix(actualresult_result, prediction_result)
print("Confusion matrix: \n",confusion_matrix)

print("Accuracy Score: ", accuracy_score(actualresult_result, prediction_result))
print("F1 Score: ", f1_score(actualresult_result, prediction_result, average='weighted'))
print("Precision Score: ", precision_score(actualresult_result, prediction_result, average='weighted'))
print("Recall Score: ", recall_score(actualresult_result, prediction_result, average='weighted'))

Processing folder:  noise
Processing folder:  good
total feature lenght:  90
['good' 'good' 'noise' 'noise' 'noise' 'good' 'good' 'good' 'good' 'good'
 'good' 'noise' 'good' 'noise' 'good' 'good' 'good' 'good' 'noise' 'good'
 'good' 'good' 'good' 'noise' 'good' 'noise' 'noise' 'good' 'good' 'good'
 'noise' 'noise' 'noise' 'good' 'good' 'good' 'noise' 'noise' 'good'
 'good' 'noise' 'good' 'noise' 'noise' 'noise' 'noise' 'noise' 'noise'
 'noise' 'noise' 'noise' 'noise' 'noise' 'noise' 'noise' 'noise' 'noise'
 'noise' 'noise' 'noise' 'good' 'good' 'good' 'good' 'good' 'good' 'good'
 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good' 'good'
 'good' 'good' 'good' 'good' 'good' 'noise' 'noise' 'good' 'good' 'good'
 'good' 'good' 'good']
Classification report: 
               precision    recall  f1-score   support

        good       0.51      0.93      0.66        30
       noise       0.94      0.55      0.69        60

    accuracy                           0.68        90
   m

In [ ]:
# This Section is to evaluate model agaisnt mixed data

jack_hammer3_file_path='102305-6-0-0-dog-bark.wav'
signal, sr = librosa.load(jack_hammer3_file_path,sr=22050,  res_type='kaiser_fast')

noise_file = 'Air_FX_01.wav'
noise_sample1, sr1 = librosa.load(noise_file, sr=None)
noise_sample = noise_sample1+noise_sample1+noise_sample1

if(len(noise_sample) > len(signal)):
    noise_sample=noise_sample[0:len(signal)]

if(len(noise_sample) < len(signal)):
    signal=signal[0:len(noise_sample)]

noise3=get_noise_from_sound(signal,noise_sample,SNR=56)
if noise3 is not None:
    signal_noise3=signal+noise3

    mfcc = extract_features_with_padding(signal_noise3, sr)
    if mfcc is not None:
        class_label = noise_file
        mfcc = np.array([mfcc])
        mfcc = np.expand_dims(mfcc, axis=3)
                    

predict = model.predict(mfcc)
predict=np.round(predict,decimals=5)
result = map(lambda v : np.argmax(v), predict)
prediction_result = np.array(list(result))
print(prediction_result)

In [ ]:
# This Section is to evaluate model agaisnt Spear data
import pandas as pd
import os
import glob
features = []

parent_dir = '/SpEAR-speech-database-master/data'
sub_dirs= ['Lombard', 'Monaural', 'Noisy_Recordings', 'TIMIT']
sub_dirs= ['noise', 'good']
file_ext = "*.wav"

file_names = []
for l, sub_dir in enumerate(sub_dirs):
    print('Processing folder: ', sub_dir)
    for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
        feature = process_sound(fn)
        file_names.append(sub_dir+'/'+fn)
        
        if sub_dir == 'confusion':
            features.append([feature, 'noise'])
            
        if sub_dir == 'Lombard':
            features.append([feature, 'noise'])
            
        if sub_dir == 'Monaural':
            features.append([feature, 'good'])
            
        if sub_dir == 'Noisy_Recordings':
            features.append([feature, 'noise'])
            
        if sub_dir == 'noise':
            features.append([feature, 'noise'])
            
        if sub_dir == 'good':
            features.append([feature, 'good'])
            
        if sub_dir == 'TIMIT':
            features.append([feature, 'good'])


featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
actualresult_result = np.array(featuresdf.class_label.tolist())
X = np.array(featuresdf.feature.tolist())
X = np.expand_dims(X, axis=3)

predict = model.predict(X)
predict=np.round(predict,decimals=5)
result = map(lambda v : np.argmax(v), predict)
prediction_result = np.array(list(result))
prediction_result = le_loaded.inverse_transform(prediction_result)

for i in range(len(prediction_result)):
    print(file_names[i] , ' : ', prediction_result[i])


classification_report = metrics.classification_report(actualresult_result, prediction_result)
print("Classification report: \n", classification_report)
confusion_matrix = metrics.confusion_matrix(actualresult_result, prediction_result)
print("Confusion matrix: \n",confusion_matrix)

print("Accuracy Score: ", accuracy_score(actualresult_result, prediction_result))
print("F1 Score: ", f1_score(actualresult_result, prediction_result, average='weighted'))
print("Precision Score: ", precision_score(actualresult_result, prediction_result, average='weighted'))
print("Recall Score: ", recall_score(actualresult_result, prediction_result, average='weighted'))